In [11]:
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", module="IPython")


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip subdirectories from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ("src",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("airquality",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("notebooks",):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir)
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH`
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from src import config

settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn
HopsworksSettings initialized!


In [12]:
import datetime
import time
import requests
import pandas as pd
import great_expectations as ge
import hopsworks
from src.airquality import util
import datetime
from pathlib import Path
import json
import re
import os
import warnings

warnings.filterwarnings("ignore")

# Setup


In [13]:
today = datetime.date.today()
project = hopsworks.login(engine="python")
fs = project.get_feature_store()

# taken from ~/.env. You can also replace settings.AQICN_API_KEY with the api key value as a string "...."
if settings.AQICN_API_KEY is None:
    print("You need to set AQICN_API_KEY either in this cell or in ~/.env")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()

print(f"Found AQICN_API_KEY: {AQICN_API_KEY}")

secrets = hopsworks.get_secrets_api()
# Replace any existing secret with the new value
try:
    secret = secrets.get_secret("AQICN_API_KEY")
    if secret is not None:
        secret.delete()
        print("Replacing existing AQICN_API_KEY")
except hopsworks.RestAPIError as e:
    # Only log the RestAPIError if it is for "not found", otherwise re-raise
    if hasattr(e, "error_code") and getattr(e, "error_code", None) == 160048:
        # Secret does not exist; proceed to create it.
        pass
    elif "Could not find Secret" in str(e):
        # Fallback check if error does not attach error_code attribute
        pass
    else:
        raise

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)


# Optional single sensor mode parameters (set in .env or leave None for batch mode)
csv_file = getattr(settings, 'SENSOR_CSV_FILE', None)
sensor_id_param = getattr(settings, 'SENSOR_ID', None)

2025-11-16 13:24:24,049 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-16 13:24:24,051 INFO: Initializing external client
2025-11-16 13:24:24,051 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-16 13:24:25,585 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279179
Found AQICN_API_KEY: 17a266deb78b8d8e7b8ad87076443a4e7ee32801
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


In [14]:
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "pm25",
            "min_value": -0.1,
            "max_value": 500.0,
            "strict_min": True,
        },
    )
)


weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)


def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column": col,
                "min_value": -0.1,
                "max_value": 1000.0,
                "strict_min": True,
            },
        )
    )


expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

# Backfill Data

## Preprocessing

In [ ]:
def read_sensor_data(file_path):
    """
    Reads the sensor data from the CSV file. The first three rows contains metadata.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        street, city, country = [
            s.strip()
            for s in f.readline()
            .strip()
            .lstrip("# Sensor ")
            .split("(")[0]
            .strip()
            .split(",")
        ]
        url_line = f.readline().strip().lstrip("# ").strip()
        sensor_id = url_line.split("@")[1].split("/")[0]
        _ = f.readline().strip()
    df = pd.read_csv(file_path, skiprows=3)
    feed_url = f"https://api.waqi.info/feed/A{sensor_id}/"
    return df, street, city, country, feed_url, sensor_id


def clean_and_append_data(df, street, city, country, feed_url, sensor_id):
    """
    Remove any unused columns, set the daily median value to pm25. Remove NaN's and append the metadata.
    """
    clean_df = pd.DataFrame()
    clean_df["date"] = pd.to_datetime(df["date"])
    clean_df["pm25"] = df["median"]
    clean_df = clean_df.dropna(subset=["pm25"])
    clean_df["sensor_id"] = sensor_id
    clean_df["street"] = street
    clean_df["city"] = city
    clean_df["country"] = country
    clean_df["feed_url"] = feed_url
    return clean_df


def get_historical_weather(city, df, today, feed_url, sensor_id):
    earliest_aq_date = pd.Series.min(df["date"])
    earliest_aq_date = earliest_aq_date.strftime("%Y-%m-%d")
    response = requests.get(f"{feed_url}/?token={AQICN_API_KEY}")
    data = response.json()
    latitude = data["data"]["city"]["geo"][0]
    longitude = data["data"]["city"]["geo"][1]
    max_retries = 5
    attempt = 0
    while attempt < max_retries:
        try:
            weather_df = util.get_historical_weather(
                city, earliest_aq_date, str(today), latitude, longitude
            )
            weather_df["sensor_id"] = sensor_id
            weather_df["city"] = city
            weather_df["latitude"] = latitude
            weather_df["longitude"] = longitude
            return weather_df, latitude, longitude
        except Exception as e:
            if hasattr(e, "args") and any(
                "Minutely API request limit exceeded" in str(a) for a in e.args
            ):
                wait_time = 70
                print(
                    f"OpenMeteo API limit exceeded, retrying in {wait_time} seconds... (Attempt {attempt + 1} of {max_retries})"
                )
                time.sleep(wait_time)
                attempt += 1
            elif "Minutely API request limit exceeded" in str(e):
                wait_time = 70
                print(
                    f"OpenMeteo API limit exceeded, retrying in {wait_time} seconds... (Attempt {attempt + 1} of {max_retries})"
                )
                time.sleep(wait_time)
                attempt += 1
            else:
                raise
    raise RuntimeError(
        "Failed to obtain historical weather after multiple retries due to API rate limits."
    )


def add_rolling_window_feature(df, window_days=3, column="pm25", new_column="pm25_rolling_3d"):
    df = df.sort_values(["sensor_id", "date"]).copy()
    
    df_indexed = df.set_index("date", append=False)
    
    df[new_column] = (
        df_indexed.groupby("sensor_id")[column]
        .rolling(window=f"{window_days}D", min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
        .values
    )
    
    return df

## Hopsworks

In [16]:
def create_air_quality_feature_group():
    air_quality_fg = fs.get_or_create_feature_group(
        name="air_quality_all",
        description="Air Quality characteristics of each day for all sensors",
        version=1,
        primary_key=["sensor_id"],
        event_time="date",
        expectation_suite=aq_expectation_suite,
    )
    return air_quality_fg


def update_air_quality_description(air_quality_fg):
    air_quality_fg.update_feature_description(
        "date", "Date of measurement of air quality"
    )
    air_quality_fg.update_feature_description(
        "sensor_id", "AQICN sensor identifier (e.g., 59893)"
    )
    air_quality_fg.update_feature_description(
        "country",
        "Country where the air quality was measured (sometimes a city in aqicn.org)",
    )
    air_quality_fg.update_feature_description(
        "city", "City where the air quality was measured"
    )
    air_quality_fg.update_feature_description(
        "street", "Street in the city where the air quality was measured"
    )
    air_quality_fg.update_feature_description(
        "pm25",
        "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk",
    )
    air_quality_fg.update_feature_description(
        "pm25_rolling_3d",
        "3-day rolling mean of PM2.5, backward filled when insufficient history available",
    )


def create_and_insert_air_quality_data(df):
    air_quality_fg = create_air_quality_feature_group()
    air_quality_fg.insert(df)
    update_air_quality_description(air_quality_fg)


def create_weather_feature_group():
    weather_fg = fs.get_or_create_feature_group(
        name="weather_all",
        description="Weather characteristics of each day for all sensors",
        version=1,
        primary_key=["sensor_id"],
        event_time="date",
        expectation_suite=weather_expectation_suite,
    )
    return weather_fg


def update_weather_description(weather_fg):
    weather_fg.update_feature_description("date", "Date of measurement of weather")
    weather_fg.update_feature_description(
        "sensor_id", "AQICN sensor identifier (e.g., 59893)"
    )
    weather_fg.update_feature_description(
        "city", "City where weather is measured/forecast for"
    )
    weather_fg.update_feature_description(
        "temperature_2m_mean", "Temperature in Celsius"
    )
    weather_fg.update_feature_description(
        "precipitation_sum", "Precipitation (rain/snow) in mm"
    )
    weather_fg.update_feature_description(
        "wind_speed_10m_max", "Wind speed at 10m abouve ground"
    )
    weather_fg.update_feature_description(
        "wind_direction_10m_dominant", "Dominant Wind direction over the dayd"
    )
    weather_fg.update_feature_description(
        "latitude", "Latitude of sensor location used for weather retrieval"
    )
    weather_fg.update_feature_description(
        "longitude", "Longitude of sensor location used for weather retrieval"
    )


def create_and_insert_weather_data(df):
    weather_fg = create_weather_feature_group()
    weather_fg.insert(df)
    update_weather_description(weather_fg)

# Script

In [17]:
all_aq_dfs = []
all_weather_dfs = []
locations = {}

if csv_file and os.path.exists(csv_file):
    # Single sensor mode if csv_file provided
    file_path = csv_file
    aq_df_raw, street, city, country, feed_url, sensor_id = read_sensor_data(file_path)
    aq_df = clean_and_append_data(aq_df_raw, street, city, country, feed_url, sensor_id)
    weather_df, latitude, longitude = get_historical_weather(
        city, aq_df, today, feed_url, sensor_id
    )
    all_aq_dfs.append(aq_df)
    all_weather_dfs.append(weather_df)
    locations[sensor_id] = {
        "country": country,
        "city": city,
        "street": street,
        "aqicn_url": feed_url,
        "latitude": latitude,
        "longitude": longitude,
    }
else:
    # Process all files in data directory
    data_dir = os.path.join(root_dir, "data")
    dir_list = os.listdir(data_dir)
    for file in dir_list:
        file_path = os.path.join(data_dir, file)
        aq_df_raw, street, city, country, feed_url, sensor_id = read_sensor_data(file_path)
        aq_df = clean_and_append_data(aq_df_raw, street, city, country, feed_url, sensor_id)
        weather_df, latitude, longitude = get_historical_weather(
            city, aq_df, today, feed_url, sensor_id
        )
        all_aq_dfs.append(aq_df)
        all_weather_dfs.append(weather_df)
        locations[sensor_id] = {
            "country": country,
            "city": city,
            "street": street,
            "aqicn_url": feed_url,
            "latitude": latitude,
            "longitude": longitude,
        }

# Concatenate into single, uniform dfs
aq_df_all = pd.concat(all_aq_dfs, ignore_index=True)
weather_df_all = pd.concat(all_weather_dfs, ignore_index=True)
aq_df_all = add_rolling_window_feature(aq_df_all, window_days=3, column="pm25", new_column="pm25_rolling_3d")

Coordinates 57.680137634277344°N 12.025862693786621°E
Elevation 20.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.680137634277344°N 12.025862693786621°E
Elevation 12.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.680137634277344°N 12.025862693786621°E
Elevation 76.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75043869018555°N 12.051836013793945°E
Elevation 51.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.680137634277344°N 12.025862693786621°E
Elevation 21.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.609840393066406°N 12.193548202514648°E
Elevation 55.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75043869018555°N 11.857451438903809°E
Elevation 7.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.680137634277344°N 12.025862693786621°E
Elevation 24.0 m asl
Timezone None None
Timezone difference 

ValueError: invalid on specified as date, must be a column (of DataFrame), an Index or None

In [ ]:
aq_df_all.info()
weather_df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28200 entries, 14524 to 4050
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   date             28200 non-null  datetime64[ns, UTC]
 1   pm25             28200 non-null  float64            
 2   sensor_id        28200 non-null  object             
 3   street           28200 non-null  object             
 4   city             28200 non-null  object             
 5   country          28200 non-null  object             
 6   feed_url         28200 non-null  object             
 7   pm25_rolling_3d  28200 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(2), object(5)
memory usage: 1.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30406 entries, 0 to 30405
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         


In [ ]:
# Create individual secrets for each sensor
for sensor_id, location in locations.items():
    secret_name = f"SENSOR_LOCATION_JSON_{sensor_id}"
    location_str = json.dumps(location)
    
    try:
        secret = secrets.get_secret(secret_name)
        if secret is not None:
            secret.delete()
            print(f"Replacing existing {secret_name}")
    except hopsworks.RestAPIError as e:
        if hasattr(e, "error_code") and getattr(e, "error_code", None) == 160048:
            pass
        elif "Could not find Secret" in str(e):
            pass
        else:
            raise
    
    secrets.create_secret(secret_name, location_str)
    print(f"Created secret: {secret_name}")

Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_59893
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_88372
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_69628
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_61714
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_60541
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_404209
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Created secret: SENSOR_LOCATION_JSON_194215
Secret created successfully, explore it at https://c.app.hopsworks.

In [ ]:
# Insert unified DataFrames into single feature groups
create_and_insert_air_quality_data(aq_df_all)
create_and_insert_weather_data(weather_df_all)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1721709
2025-11-16 13:18:09,828 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1721709


Uploading Dataframe: 100.00% |██████████| Rows 28200/28200 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_all_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279179/jobs/named/air_quality_all_1_offline_fg_materialization/executions
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1721710
2025-11-16 13:18:32,747 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1721710


Uploading Dataframe: 100.00% |██████████| Rows 30406/30406 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_all_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279179/jobs/named/weather_all_1_offline_fg_materialization/executions
